In [324]:
import pandas as pd
from datetime import datetime
import numpy as np
import json
import glob
import keras
from datetime import timedelta
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import json
import seaborn as sns
%matplotlib inline

- Clean notebook to be only new 
- Be able to merge train features and train targets
- same with test
- be able to then split and run program
- test for overfit
- create additional features (business/extra features from swing/ extra features from bugs)
- run reviews for NLP

In [325]:
list_of_file_names = glob.glob("./zip_2/*.csv")

In [326]:
read_data_list_2 = []
fail_list = []
for i, data in enumerate(list_of_file_names):
    try:
        data = pd.read_csv(data)
        #data = unicode(data, errors='replace')
        read_data_list_2.append(data)
    except UnicodeDecodeError:
        fail_list.append(i)
#read_data_list
#fail_list
list_of_file_names_org = list_of_file_names

In [327]:
fail_list

[3]

In [328]:
for x in fail_list:
    del list_of_file_names[x]

In [329]:
list_of_file_names[3]

'./zip_2/Zip_Listings_PriceCut_SeasAdj_AllHomes.csv'

In [330]:
# build loop for data we can work with. Only dates and zip
def build_useful_df(data_list):
    useful_df = []
    for dataset in range(len(data_list)):
        data = read_data_list_2[dataset]
        region_name = data['RegionName']
        region_name = region_name.astype(str)
        new_df = data.select_dtypes(include=['float64'])
        new_df.insert(loc=0, column='RegionName', value=region_name) 
        new_df = new_df.sort_values('RegionName',ascending=False)
        new_df = new_df.set_index('RegionName')
        useful_df.append(new_df)
    return useful_df

In [331]:
useful_df = build_useful_df(read_data_list_2)


In [332]:
sixteen_list = []
for i, file in enumerate(useful_df):
    if len(file) >= 10000:
        sixteen_list.append(i)

In [333]:
len(sixteen_list)

30

In [334]:
good_columns = []
for i, file in enumerate(useful_df):
    columns = file.columns
    if '2017-01' in columns:
        good_columns.append(i)

In [335]:
sixteen_good = []
for num in sixteen_list:
    if num in good_columns:
        sixteen_good.append(num)

In [336]:
len(sixteen_good)

12

In [320]:
len(useful_df)
for num in range(len(useful_df)):
    useful_df[num].dropna(inplace=True)

In [337]:
target_data = useful_df[23]

In [338]:
final_data_list = []
for i, data in enumerate(useful_df):
    if i in sixteen_good:
        final_data_list.append(data)

In [339]:
len(final_data_list[4])

12960

In [340]:
# Docs that don't need feature engineering. They are already aggregates
pure_features = [0, 71, 75, 77, 79, 81]
useful_df[0].head()

,PctPriceCut,DaysOnMarket,BuyerSellerIndex,BuyerSellerIndexMetro
RegionName,,,,
99654,11.142857,82.0,9.230769,7.260083
99645,7.589286,74.0,0.769231,7.260083
99577,8.108108,79.5,4.615385,7.260083
99567,9.836066,76.0,2.307692,7.260083
99518,14.925373,85.0,10.000000,7.260083


In [383]:
def make_features(df, past_time_string, now_string):
    #df.dropna(inplace=True)
    features = pd.DataFrame()
    #features['RegionName'] = df['RegionName']
    mean = df.loc[:, past_time_string : now_string].mean(axis=1)
    features['mean'] = mean
    std = df.loc[:, past_time_string : now_string].std(axis=1)
    features['std'] = std
    mn = df.loc[:, past_time_string : now_string].min(axis=1)
    features['min'] = mn
    mx = df.loc[:, past_time_string : now_string].max(axis=1)
    features['max'] = mx
    features['swing'] = mx - mn
    change = df[now_string] - df[past_time_string]
    features['change'] = change
    mean_swing = features['swing'].mean()
    features['swing_pos'] = np.where(features['swing']>mean_swing, 1, 0)
    big_swing = features['swing'].std() + mean_swing
    features['swing_big'] = np.where(features['swing']>big_swing, 1, 0)
    features['swing_neg'] = np.where(features['swing']<mean_swing, 1, 0)
    swing_big_loss = mean_swing - features['swing'].std() 
    features['swing_loss_big'] = np.where(features['swing']<swing_big_loss, 1, 0)
    #features.dropna(inplace=True)
    
    #features = features.set_index(df.index)
    #print(features.iloc[1, :])
    return features
    
# List of data frames only on one now_time
def make_modeling_data(df_list, df_for_target, now_string):
    df_one = pd.DataFrame()
    target = pd.DataFrame()
    now_time = pd.to_datetime(now_string)
    now_value = df_for_target[now_string]
    future_time = now_time + timedelta(days=6*31)
    future_time_string= future_time.strftime("%Y-%m")
    future_value = df_for_target[future_time_string]
    target['target'] = future_value/now_value
    
    
    past_time = now_time - timedelta(days=6*31)
    past_time_string= past_time.strftime("%Y-%m")
    df_to_use_for_features_org= df_list[0].loc[:, :now_string]
    features_org = make_features(df_to_use_for_features_org, past_time_string, now_string)
    df_one = pd.merge(df_one, features_org, left_index=True, right_index=True, how = 'right')
    for i, df in enumerate(df_list[1:]):
        ind = str(i)
        columns = df.columns
        if '2011-01' in columns and '2012-01' in columns and '2013-01' in columns and '2014-01' in columns and '2015-01' in columns and '2016-01' in columns and '2017-01' in columns:
            df_to_use_for_features= df.loc[:, :now_string]
            features = make_features(df_to_use_for_features, past_time_string, now_string)
            df_one = pd.merge(df_one, features, right_index=True, left_index=True, how='inner')
    target = target.loc[df_one.index]    
    #now_time = pd.to_datetime(now_string)
    #now_value = df_for_target[now_string]
    #future_time = now_time + timedelta(days=6*31)
    #future_time_string= future_time.strftime("%Y-%m")
    #future_value = df_for_target[future_time_string]
            #target = future_value/now_value

    
    return df_one, target




test_features, test_targets = make_modeling_data(final_data_list, target_data, "2017-06")
train_features, train_targets = make_modeling_data(final_data_list, target_data, "2016-12")
extra_train_features_1, extra_train_targets_1 = make_modeling_data(final_data_list, target_data, '2016-06')
train_features = train_features.append(extra_train_features_1)
train_targets = train_targets.append(extra_train_targets_1)






#df = useful_df[num]
#columns = df.columns
#"2012", "2012", "2013",
date_counter = 0
for year in ["2012", "2013", "2014", "2015"]:
    for month in ["06","12"]:
            new_time = year+"-"+month
            date_counter += 1
            #if '2014-01' in columns and '2015-01' in columns and '2016-01' in columns and '2017-01' in columns:
                    #extra_train_features, extra_train_targets = make_modeling_data(df, new_time)
                    #train_features = train_features.append(extra_train_features)
                    #train_targets = train_targets.append(extra_train_targets)
            extra_train_features, extra_train_targets = make_modeling_data(final_data_list, target_data, new_time)
            #extra_train_features.dropna(inplace=True)
            train_features = train_features.append(extra_train_features)
            train_targets = train_targets.append(extra_train_targets)

In [384]:
train_targets.shape

(55110, 1)

In [386]:
train_features.shape

(55110, 120)

In [170]:
test_features.to_csv('test_features_to_merge_final.csv')

In [385]:
train_targets.head()

,target
RegionName,
99362,1.053118
99353,1.087944
99352,1.068541
99338,1.099950
99337,1.056829


#### Have not worked on w2v today; focused on business

In [115]:
w2v_features = pd.read_csv('w2v_for_features')

In [41]:
w2v_features['zips'] = w2v_features['zips'].astype(str)

In [46]:
w2v_features.shape

(55, 501)

In [42]:
w2v_features.index = w2v_features['zips']

In [43]:
w2v_features = w2v_features.iloc[:, 1:]

In [44]:
w2v_features.shape

(55, 500)

#### Here is where business is imported

In [387]:
# Up to date business features where all rows are a singular zipcode
business_features = pd.read_csv('real_business_features_3_4')

In [388]:
len(business_features)

959

In [389]:
business_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 11 columns):
postal_code             959 non-null int64
is_open                 959 non-null float64
latitude                959 non-null float64
longitude               959 non-null float64
review_count            959 non-null float64
stars                   959 non-null float64
average_review_count    959 non-null float64
business_count          959 non-null float64
zip_review_count        959 non-null float64
average_review_max      959 non-null float64
average_review_min      959 non-null float64
dtypes: float64(10), int64(1)
memory usage: 82.5 KB


In [390]:
business_features.shape

(959, 11)

In [391]:
business_features['postal_code'] = business_features['postal_code'].astype(str)

In [392]:
business_features['postal_code'].nunique()

959

In [393]:
business_features = business_features.set_index('postal_code')

In [394]:
business_features.head()

,is_open,latitude,longitude,review_count,stars,average_review_count,business_count,zip_review_count,average_review_max,average_review_min
postal_code,,,,,,,,,,
2224,1.0,41.499208,-81.536689,7.000000,2.500000,7.000000,1.0,7.0,7.0,7.0
5440,1.0,44.964078,-73.285549,4.000000,4.250000,4.000000,2.0,8.0,5.0,3.0
5452,1.0,40.780821,-74.150722,49.000000,4.000000,49.000000,1.0,49.0,49.0,49.0
6618,1.0,51.083200,11.858200,8.000000,3.500000,8.000000,1.0,8.0,8.0,8.0
6632,1.0,51.213256,11.763245,5.888889,4.222222,5.888889,9.0,53.0,8.0,3.0


In [395]:
train_features.shape

(55110, 120)

In [396]:
train_features = train_features.reset_index()

In [397]:
test_features = test_features.reset_index()

In [398]:
len(test_features)

5511

In [399]:
train_targets = train_targets.reset_index()

In [400]:
test_targets = test_targets.reset_index()

In [401]:
business_features_index = business_features.index.unique()

In [402]:
region_name_list = train_features['RegionName'].unique()

In [403]:
shared_list = []
for zipc in business_features_index:
    if zipc in region_name_list:
        shared_list.append(zipc)

In [404]:
len(shared_list)

333

In [405]:
len(train_features)

55110

In [406]:
# Merging on index for train features and targets so it doesn't multiply 
train_merge = pd.merge(train_features, train_targets, right_index=True, left_index=True, how='inner')

In [407]:
test_merge = pd.merge(test_features, test_targets, right_index=True, left_index=True, how='inner')

In [408]:
len(train_merge)

55110

In [409]:
test_merge.head()

,RegionName_x,mean_x,std_x,min_x,max_x,swing_x,change_x,swing_pos_x,swing_big_x,swing_neg_x,...,min_y,max_y,swing_y,change_y,swing_pos_y,swing_big_y,swing_neg_y,swing_loss_big_y,RegionName_y,target
0,99362,9.899305,2.694838,7.225547,15.562721,8.337174,-8.337174,0,0,1,...,0.874,0.894,0.020,0.018,0,0,1,0,99362,1.000168
1,99353,12.713502,4.201132,8.701869,20.788350,12.086481,6.728607,1,1,0,...,0.874,0.886,0.012,-0.010,0,0,1,0,99353,1.137133
2,99352,13.114296,2.222078,10.907130,17.667674,6.760545,2.172499,0,0,1,...,0.900,0.908,0.008,-0.004,0,0,1,0,99352,1.011334
3,99338,10.337470,2.859324,6.462035,15.113417,8.651382,-1.678754,1,0,0,...,0.884,0.896,0.012,0.008,0,0,1,0,99338,1.014762
4,99337,12.309892,3.922309,7.724349,18.444138,10.719789,-6.511823,1,0,0,...,0.878,0.890,0.012,0.012,0,0,1,0,99337,1.062254


In [410]:
train_merge = train_merge.set_index('RegionName_x')

In [411]:
test_merge = test_merge.set_index('RegionName_x')

In [412]:
train_merge.shape

(55110, 122)

In [413]:
train_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55110 entries, 99362 to 1001
Columns: 122 entries, mean_x to target
dtypes: float64(73), int64(48), object(1)
memory usage: 51.7+ MB


In [414]:
business_features.shape

(959, 10)

In [415]:
train_merge['RegionName_y'].nunique()

5511

In [416]:
business_train_merge = pd.merge(train_merge, business_features, right_index=True, left_index=True, how='inner')

In [417]:
business_test_merge = pd.merge(test_merge, business_features, right_index=True, left_index=True, how='inner')

In [418]:
# This should have all instances of the train and test. 
business_train_merge.shape

(3330, 132)

In [303]:
# Is it ok that I have multiple of the aggregated information from Yelp?
business_train_merge.head(11)

,mean_x,std_x,min_x,max_x,swing_x,change_x,swing_pos_x,swing_big_x,swing_neg_x,swing_loss_big_x,...,is_open,latitude,longitude,review_count,stars,average_review_count,business_count,zip_review_count,average_review_max,average_review_min
15003,14.181132,2.147629,11.537536,18.258360,6.720823,-1.803249,0,0,1,0,...,0.916667,40.598250,-80.223206,10.861111,3.958333,10.861111,36.0,391.0,50.0,3.0
15003,15.011149,3.868854,8.586176,21.331859,12.745683,-3.823192,1,1,0,0,...,0.916667,40.598250,-80.223206,10.861111,3.958333,10.861111,36.0,391.0,50.0,3.0
15003,13.066543,2.749281,9.918464,18.321159,8.402695,-1.485973,1,0,0,0,...,0.916667,40.598250,-80.223206,10.861111,3.958333,10.861111,36.0,391.0,50.0,3.0
15003,14.267149,3.381999,9.615653,19.912455,10.296802,-3.850137,1,0,0,0,...,0.916667,40.598250,-80.223206,10.861111,3.958333,10.861111,36.0,391.0,50.0,3.0
15003,15.460086,3.876415,9.615653,20.093610,10.477957,8.783796,1,0,0,0,...,0.916667,40.598250,-80.223206,10.861111,3.958333,10.861111,36.0,391.0,50.0,3.0
15003,16.248923,2.768642,11.968512,21.010779,9.042267,2.144157,1,0,0,0,...,0.916667,40.598250,-80.223206,10.861111,3.958333,10.861111,36.0,391.0,50.0,3.0
15003,16.116125,4.855846,8.884885,24.155285,15.270400,2.256744,1,1,0,0,...,0.916667,40.598250,-80.223206,10.861111,3.958333,10.861111,36.0,391.0,50.0,3.0
15003,16.885622,3.206663,13.813915,24.155285,10.341370,-8.362070,1,0,0,0,...,0.916667,40.598250,-80.223206,10.861111,3.958333,10.861111,36.0,391.0,50.0,3.0
15003,13.163923,2.375110,9.706724,16.074128,6.367404,-2.375702,0,0,1,0,...,0.916667,40.598250,-80.223206,10.861111,3.958333,10.861111,36.0,391.0,50.0,3.0
15003,14.980037,2.924676,9.706724,19.470481,9.763757,8.018858,1,0,0,0,...,0.916667,40.598250,-80.223206,10.861111,3.958333,10.861111,36.0,391.0,50.0,3.0


In [323]:
X_train = train_merge.iloc[:, :-1].values
y_train = train_merge.iloc[:, -1].values
X_test = test_merge.iloc[:, :-1].values
y_test = test_merge.iloc[:, -1].values

### Most up to date Business attempt
- Why is y glitching with NaN?
- Update I have not removed any NaN's yet

In [419]:
business_train_merge.dropna(inplace=True)

In [420]:
business_test_merge.dropna(inplace=True)

In [421]:
# 169 zipcodes in common after dropna. Had about double that before
business_test_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331 entries, 95966 to 15003
Columns: 132 entries, mean_x to average_review_min
dtypes: float64(83), int64(48), object(1)
memory usage: 343.9+ KB


In [422]:
# Setting up train and test
business_train_merge = business_train_merge.drop('RegionName_y', 1)
business_test_merge = business_test_merge.drop('RegionName_y', 1)
y_train_business = business_train_merge.loc[:, 'target'].values
y_test_business = business_test_merge.loc[:, 'target'].values
business_train = pd.DataFrame()
business_train_features = business_train_merge.drop('target', 1).values
X_train_business = pd.DataFrame()
X_train_business = business_train_features
X_test_business_features = pd.DataFrame()
X_test_business_features = business_test_merge.drop('target', 1).values
X_test_business = pd.DataFrame()
X_test_business = X_test_business_features

In [423]:
X_test_business.shape

(331, 130)

In [399]:
#For w2v import
X_train_w2v = train_merge_w2v.iloc[:, :-1].values
y_train_w2v = train_merge_w2v.iloc[:, -1].values
X_test_w2v = test_merge_w2v.iloc[:, :-1].values
y_test_w2v = test_merge_w2v.iloc[:, -1].values

In [400]:
X_train_w2v.shape

(5400, 610)

In [424]:
def norm_per_column(X):
    new_array = []
    shape = X.shape
    columns = shape[1]
    for num in range(columns):
        mx = X[:, num].max()
        if mx == 0:
            new_array.append(X[:, num])
        else:
            X[:, num] = X[:, num] / mx
            new_array.append(X[:, num])
    new_array = np.array(new_array)
    new_array = np.transpose(new_array)
    return new_array

In [327]:
X_train = norm_per_column(X_train)
X_test = norm_per_column(X_test)

In [425]:
X_train_business = norm_per_column(X_train_business)
X_test_business = norm_per_column(X_test_business)

In [401]:
X_train_w2v = norm_per_column(X_train_w2v)
X_test_w2v = norm_per_column(X_test_w2v)

In [291]:
y_train_business.shape

(169, 70)

In [62]:
# Hold overfit model for now
model = Sequential()
model.add(Dense(3000, input_shape=(110,), activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
sgd = keras.optimizers.SGD(lr=0.75)
model.compile(optimizer='Adam',
              loss='mean_squared_error')

In [81]:
model = Sequential()
model.add(Dense(1000, input_shape=(110,), activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
sgd = keras.optimizers.SGD(lr=0.75)
model.compile(optimizer='Adam',
              loss='mean_squared_error')

In [83]:
model.fit(x=X_train, y=y_train, 
          batch_size=5000, 
          epochs=40, 
          verbose=1, 
          validation_data=(X_test, y_test),
          callbacks=None)

Train on 126808 samples, validate on 1048 samples
Epoch 1/40
126808/126808 [==============================] - 31s 247us/step - loss: 0.0025 - val_loss: 0.0050
Epoch 2/40
126808/126808 [==============================] - 31s 244us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 3/40
126808/126808 [==============================] - 31s 244us/step - loss: 0.0025 - val_loss: 0.0052
Epoch 4/40
126808/126808 [==============================] - 31s 244us/step - loss: 0.0025 - val_loss: 0.0049
Epoch 5/40
126808/126808 [==============================] - 31s 245us/step - loss: 0.0025 - val_loss: 0.0042
Epoch 6/40
126808/126808 [==============================] - 31s 247us/step - loss: 0.0025 - val_loss: 0.0056
Epoch 7/40
126808/126808 [==============================] - 31s 246us/step - loss: 0.0025 - val_loss: 0.0073
Epoch 8/40
126808/126808 [==============================] - 31s 245us/step - loss: 0.0025 - val_loss: 0.0057
Epoch 9/40
126808/126808 [==============================] - 31s 244us/step - l

In [ ]:
#loss = .00001441 (overfit) = .0038

#std = .072

In [428]:
model = Sequential()
model.add(Dense(3000, input_shape=(130,), activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
sgd = keras.optimizers.SGD(lr=0.75)
model.compile(optimizer='Adam',
              loss='mean_squared_error')

In [429]:
model.fit(x=X_train_business, y=y_train_business, 
          batch_size=5000, 
          epochs=40, 
          verbose=1, 
          validation_data=(X_test_business, y_test_business),
          callbacks=None)

Train on 2187 samples, validate on 331 samples
Epoch 1/40
2187/2187 [==============================] - 4s 2ms/step - loss: 1.2322 - val_loss: 2.1528
Epoch 2/40
2187/2187 [==============================] - 3s 1ms/step - loss: 1.9971 - val_loss: 0.2972
Epoch 3/40
2187/2187 [==============================] - 3s 1ms/step - loss: 0.3186 - val_loss: 0.0314
Epoch 4/40
2187/2187 [==============================] - 3s 1ms/step - loss: 0.0320 - val_loss: 0.1145
Epoch 5/40
2187/2187 [==============================] - 3s 1ms/step - loss: 0.1030 - val_loss: 0.1072
Epoch 6/40
2187/2187 [==============================] - 3s 1ms/step - loss: 0.1235 - val_loss: 0.0181
Epoch 7/40
2187/2187 [==============================] - 3s 1ms/step - loss: 0.0236 - val_loss: 0.1243
Epoch 8/40
2187/2187 [==============================] - 3s 1ms/step - loss: 0.1099 - val_loss: 0.0258
Epoch 9/40
2187/2187 [==============================] - 3s 1ms/step - loss: 0.0352 - val_loss: 0.0521
Epoch 10/40
2187/2187 [============

In [403]:
model = Sequential()
model.add(Dense(3000, input_shape=(610,), activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
sgd = keras.optimizers.SGD(lr=0.75)
model.compile(optimizer='Adam',
              loss='mean_squared_error')

In [405]:
model.fit(x=X_train_w2v, y=y_train_w2v, 
          batch_size=5000, 
          epochs=40, 
          verbose=1, 
          validation_data=(X_test_w2v, y_test_w2v),
          callbacks=None)

Train on 5400 samples, validate on 55 samples
Epoch 1/40
5400/5400 [==============================] - 7s 1ms/step - loss: 0.0039 - val_loss: nan
Epoch 2/40
5400/5400 [==============================] - 7s 1ms/step - loss: 0.0038 - val_loss: nan
Epoch 3/40
5400/5400 [==============================] - 7s 1ms/step - loss: 0.0038 - val_loss: nan
Epoch 4/40
5400/5400 [==============================] - 7s 1ms/step - loss: 0.0037 - val_loss: nan
Epoch 5/40


KeyboardInterrupt: 